# Tensorflow MNIST Pixel Threshold demo

This notebook contains a demonstration of how to use Dioptra to run experiments that investigate the effects of the pixel threshold attack when launched on a neural network model trained on the MNIST dataset.

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [ ]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Path to custom task plugins archives
CUSTOM_PLUGINS_EVALUATION_TAR_GZ = Path("custom-plugins-evaluation.tar.gz"),

# Path to custom task plugins archives
CUSTOM_PLUGINS_PIXEL_THRESHOLD_TAR_GZ = Path("custom-plugins-pixel-threshold.tar.gz"),

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = "jtsexton_mnist_pixel_threshold"

# Path to dataset
data_path_mnist = "/nfs/data/Mnist"

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [ ]:
restapi_client = utils.SecuringAIClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Lab API and returns a list of active queues.

In [ ]:
restapi_client.list_queues()

This example also makes use of the `evaluation` custom task plugin package stored locally under the `task-plugins/securingai_custom/evaluation` directory, as well as the `pixel_threshold` custom task plugin packages stored locally under the `task-plugins/securingai_custom/pixel_threshold`.
To register these custom task plugins, we first need to package them up into an archive.
For convenience, the `Makefile` provides a rule for creating the custom task plugins archive file, just run `make custom-plugins`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins

Now that the `evaluation` and `pixel_threshold` custom task plugin packages are packaged into archive files, next we register them by uploading the files to the REST API.
Note that we need to provide the name to use for custom task plugin package, this name must be unique under the custom task plugins namespace.
For a full list of the custom task plugins, use `restapi_client.restapi_client.list_custom_task_plugins()`.

In [ ]:
response_custom_plugins = restapi_client.get_custom_task_plugin(name="evaluation")
#restapi_client.delete_custom_task_plugin(name="evaluation")
if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="evaluation",
        custom_plugin_file=CUSTOM_PLUGINS_EVALUATION_TAR_GZ,
    )

response_custom_plugins

response_custom_plugins = restapi_client.get_custom_task_plugin(name="pixel_threshold")
#restapi_client.delete_custom_task_plugin(name="pixel_threshold")
if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="pixel_threshold",
        custom_plugin_file=CUSTOM_PLUGINS_PIXEL_THRESHOLD_TAR_GZ,
    )

response_custom_plugins

If at any point you need to update one or more files within the `evaluation` and `pixel_threshold` plugin packages, you will need to unregister/delete the custom task plugin first using the REST API.
This can be done as follows,

```python
# Delete the 'evaluation' custom task plugin package
restapi_client.delete_custom_task_plugin(name="evaluation")
restapi_client.delete_custom_task_plugin(name="pixel_threshold")
```

Next, we need to create our model.
In this example, we will be creating an MNIST classifier. So, we use the ```train``` entry point to simply train a new MNIST model.

In [ ]:
response_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        f"-P data_dir={data_path_mnist}",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

pprint.pprint(response_train)

Now that we have an MNIST trained model, we can run the Pixel Threshold attack on it.
The Pixel Threshold attack attempts to change a limited number of pixels in a test image in an attempt to get it misclassified.
It has two main arguments:

| parameter | data type | description |
| --- | --- | --- |
| th | int | The maximum number of pixels it is allowed to change |
| `es` | int | If 0, then use the CMA-ES strategy, or if 1, use the DE strategy for evolution |

In [ ]:
def mlflow_run_id_is_not_known(response_pt):
    return response_pt["mlflowRunId"] is None and response_pt["status"] not in [
        "failed",
        "finished",
    ]

response_pt = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="pt",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name=jtsexton_mnist_le_net",
            "-P model_version=1",
            f"-P data_dir={data_path_mnist}",
            "-P batch_size=32",
            "-P th=1",
            "-P es=0"
        ]
    ),
    queue="tensorflow_gpu",
    timeout="1h"
)

pprint.pprint(response_pt)